supply and demand analysis
calculate changes in oil reserves, including the SPR, weekly and compared to last year
determine if changes are in line with expectations
if not in line with expectations determine if it is do to higher demand, lower production, more exports or a combination

In [1]:
import pandas as pd
import USA_oil_data as oil
import prices
import FRED_oil_data as fred
import matplotlib.pyplot as plt

In [2]:
weekly_stocks = oil.weekly_stocks()
spr = oil.spr_reserves()